<a href="https://colab.research.google.com/github/anwishn/ESAA/blob/main/1121_%ED%9A%8C%EA%B7%801_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 단순 선형 회귀 분석**
- 전복의 나이를 예측하는 선형회귀모델을 생성하세요.
- 전복의 ‘성별’, ‘키’, ‘지름’, ‘높이’, ‘전체무게’, ‘몸통무게’, ‘내장무게’, ‘껍질무게’를 이용해 ‘껍질의 고리 수’를 예측한 뒤, **예측된 ‘껍질의 고리 수’에 1.5를 더하면 전복의 나이**가 됩니다.

In [57]:
# 기본 모듈 불러오기
import numpy as np
import pandas as pd

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**1) 데이터 load 및 변형** 

In [59]:
# 데이터 로드
data = pd.read_csv("/content/drive/MyDrive/data/abalone.csv")
print(data.head())
print(data.shape)

# 성별 M은 Male, F는 Female, I는 Infant 이므로 따로 열 만들기
for label in "MFI":
    data[label] = data["Sex"] == label
data.drop('Sex', axis=1, inplace=True)

  Sex  Length  Diameter  Height  Whole weight  Shucked weight  Viscera weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7  
(4177, 9)


**2) X, y 선택**
: y는 Rings열, X는 Rings열을 제외한 나머지를 선택하되 전부 실수가 되도록 한다.

In [60]:
# X,y 데이터 선택
y = data['Rings'].astype('float')

X = data.drop('Rings',axis=1).astype('float')

 **3) train/test set 분리**

In [61]:
# 필요한 모듈 불러오기
from sklearn.model_selection import train_test_split

In [62]:
# train과 test set 분리 (train:test = 7:3 비율로)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=156)

**4) 선형회귀모델 생성, 모델 예측치 구하기**

In [63]:
#필요한 모듈 불러오기
from sklearn.linear_model import LinearRegression

In [64]:
#선형회귀모델 생성 및 훈련
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [65]:
# 모델 예측치 구하기
y_preds = lr.predict(X_test)
# 모델 예측치를 활용해 최종적으로 전복의 나이를 예측
print('age:',y_preds+1.5)

age: [12.93605621 10.31680051 10.05943354 ... 12.48281237 10.79055844
  7.88127031]


**5) 모델 평가: MSE, RMSE, R2 score, corr 구하기**

In [66]:
#필요한 모듈 불러오기
from sklearn.metrics import mean_squared_error, r2_score

- MSE, RMSE 

In [67]:
#mse, rmse
mse = mean_squared_error(y_test,y_preds)
rmse = np.sqrt(mse)
print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse,rmse))

MSE : 5.193, RMSE : 2.279


- R2 score

In [68]:
#R2 score 측정
print(np.round(r2_score(y_test,y_preds),3))

0.486


- 회귀 절편값

In [69]:
#회귀 절편 값
print('절편 값:',lr.intercept_)

절편 값: 3.248395545002488


- 회귀 계수 값

In [70]:
#회귀 계수 값
print('회귀 계수값:',np.round(lr.coef_,1))

회귀 계수값: [ -2.7  11.2  24.2   9.6 -20.2 -11.6   6.6   0.3   0.2  -0.5]


- 상관계수

Hint: corr 함수 이용.

In [71]:
# 상관계수 구하기
df = pd.DataFrame({'y_test':y_test,'y_preds':y_preds})
corr = df.corr()
print(corr)

           y_test   y_preds
y_test   1.000000  0.701207
y_preds  0.701207  1.000000


# **2. Polynomial features**

In [72]:
# PolynomialFeatures 라이브러리 호출
from sklearn.preprocessing import PolynomialFeatures

In [73]:
# 임의 데이터 생성

X = np.arange(6).reshape(3, 2)

df =  pd.DataFrame(X)
df.columns = ['x_1','x_2']
df

,x_1,x_2
0,0,1
1,2,3
2,4,5


In [74]:
# 차원은 2로 설정
# fit_transform 메소드를 통해 데이터 변환
# PolynomialFeatures로 변환 된 데이터를 데이터 프레임 형태로 변환
poly_ftr = PolynomialFeatures(degree=2).fit_transform(X)
df_poly = pd.DataFrame(poly_ftr)

In [75]:
df_poly

,0,1,2,3,4,5
0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,2.0,3.0,4.0,6.0,9.0
2,1.0,4.0,5.0,16.0,20.0,25.0


In [76]:
# df_poly의 컬럼을 1,x1,x2,x1^2,x1*x2,x2^2 로 변경
df_poly.columns = ['1','x1','x2','x1^2','x1*x2','x2^2']

In [77]:
df_poly

,1,x1,x2,x1^2,x1*x2,x2^2
0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,2.0,3.0,4.0,6.0,9.0
2,1.0,4.0,5.0,16.0,20.0,25.0
